Задание 1.
Задание: обучите три классификатора:

1) на токенах с высокой частотой

2) на токенах со средней частотой

3) на токенах с низкой частотой

Сравните полученные результаты, оцените какие токены наиболее важные для классификации.

In [186]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
import os

In [187]:
from google.colab import drive
drive.mount('/content/drive')

pos = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Text/positive.csv', on_bad_lines='skip', sep = ';', header=None)
pos.head(3)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,0,1,2,3,4,5,6,7,8,9,10,11
0,408906692374446080,1386325927,pleease_shut_up,"@first_timee хоть я и школота, но поверь, у на...",1,0,0,0,7569,62,61,0
1,408906692693221377,1386325927,alinakirpicheva,"Да, все-таки он немного похож на него. Но мой ...",1,0,0,0,11825,59,31,2
2,408906695083954177,1386325927,EvgeshaRe,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...,1,0,1,0,1273,26,27,0


In [188]:
pos["label"] = 1

In [189]:
neg = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Text/negative.csv', on_bad_lines='skip', sep = ';', header=None)
neg.head(3)

,0,1,2,3,4,5,6,7,8,9,10,11
0,408906762813579264,1386325944,dugarchikbellko,на работе был полный пиддес :| и так каждое за...,-1,0,0,0,8064,111,94,2
1,408906818262687744,1386325957,nugemycejela,"Коллеги сидят рубятся в Urban terror, а я из-з...",-1,0,0,0,26,42,39,0
2,408906858515398656,1386325966,4post21,@elina_4post как говорят обещаного три года жд...,-1,0,0,0,718,49,249,0


In [190]:
neg["label"] = 0

In [191]:
df = pd.concat([pos, neg], axis = 0, sort = False)
df.head(3)

,0,1,2,3,4,5,6,7,8,9,10,11,label
0,408906692374446080,1386325927,pleease_shut_up,"@first_timee хоть я и школота, но поверь, у на...",1,0,0,0,7569,62,61,0,1
1,408906692693221377,1386325927,alinakirpicheva,"Да, все-таки он немного похож на него. Но мой ...",1,0,0,0,11825,59,31,2,1
2,408906695083954177,1386325927,EvgeshaRe,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...,1,0,1,0,1273,26,27,0,1


In [192]:
df.value_counts('label')

label
0    9
1    8
dtype: int64

In [193]:
data = pd.DataFrame(columns = ['text', 'label'])
data['text'] = df[3]
data['label'] = df['label']
data.head(3)

,text,label
0,"@first_timee хоть я и школота, но поверь, у на...",1
1,"Да, все-таки он немного похож на него. Но мой ...",1
2,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...,1


In [194]:
data.to_csv('/content/drive/My Drive/Colab Notebooks/Text/data.csv', index= False )

In [195]:
#очистим от имен пользователей и ссылок на телеграм
data['text'] = data['text'].apply(lambda x: re.sub("http://[\w]*", ' ', x))
data['text'] = data['text'].apply(lambda x: re.sub("@[\w]*", ' ', x))
data['text'] = data['text'].apply(lambda x: re.sub(r'[^\w\s]', ' ', x))
data['text'] = data['text'].apply(lambda x: x.lower())
data['text'] = data['text'].apply(lambda x: re.sub(r'[^а-яА-я0-9]', ' ', x))
data['text'] = data['text'].apply(lambda x: re.sub(r'[^а-яА-я]', ' ', x))

In [196]:
data['text'] = data['text'].apply(lambda x: x.join([y for y in x.split(" ") if len(y)>1]))
data.head(3)

,text,label
0,хоть хоть я и школота но поверь у нас то же...,1
1,дада все таки он немного похож на него но мо...,1
2,ну ну ты идиотка я испугалась за тебя ...,1


In [197]:
data.to_csv('/content/drive/My Drive/Colab Notebooks/Text/data_clean.csv', index= False )

In [198]:
data.head(10)

,text,label
0,хоть хоть я и школота но поверь у нас то же...,1
1,дада все таки он немного похож на него но мо...,1
2,ну ну ты идиотка я испугалась за тебя ...,1
3,кто кто то в углу сидит и погибает от го...,1
4,вот вот что значит страшилка но блин посм...,1
5,нуну любишь или нет я не знаю кто ты бля ...,1
6,ох ох ну это конечно же чтобы ...,1
7,тебя у тебя есть ухаж р нет мои уши не...,1
0,нана работе был полный пиддес и так каждое ...,0
1,коллегиколлеги сидят рубятся в а...,0


In [199]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

tfidf_vectorizer = TfidfVectorizer()
values = tfidf_vectorizer.fit_transform(data['text'])

# Show the Model as a pandas DataFrame
feature_names = tfidf_vectorizer.get_feature_names_out()
data_vec = pd.DataFrame(values.toarray(), columns = feature_names)
data_vec.head(5)

,авторы,блин,бля,болею,болеюа,буду,будужелаю,был,былна,было,...,хорошего,хорошегожелаю,хотим,хоть,хотя,части,что,чтобы,школота,это
0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.306316,0.000000,0.000000,0.000000,0.0,0.306316,0.0
1,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0
2,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0
3,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.256693,0.000000,0.256693,0.000000,0.000000,0.0,0.000000,0.0
4,0.216067,0.216067,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.000000,0.216067,0.648202,0.0,0.000000,0.0


In [200]:
mean_weights = np.asarray(values.mean(axis = 0)).ravel().tolist()
print(mean_weights)

# создаём датафрейм из словаря
mean_weights_df = pd.DataFrame({'term': tfidf_vectorizer.get_feature_names_out(), 'mean_weights': mean_weights})

mean_weights_df.to_csv('/content/drive/My Drive/Colab Notebooks/Text/mean_weights_df.csv', index= False )

[0.012709845912627776, 0.012709845912627776, 0.025195143295750146, 0.025273884977619447, 0.006318471244404862, 0.019241797875480367, 0.002137977541720041, 0.016954307098411187, 0.0015413006453101077, 0.016083860817592173, 0.016083860817592173, 0.015099559152917138, 0.02166218665672703, 0.002707773332090879, 0.025273884977619447, 0.006318471244404862, 0.012709845912627776, 0.03887822172629392, 0.002447300689112766, 0.02302334375482737, 0.003837223959137895, 0.03159235622202431, 0.024014605321403704, 0.024014605321403704, 0.015099559152917138, 0.001223650344556383, 0.02166218665672703, 0.002707773332090879, 0.016650712285423714, 0.016083860817592173, 0.02021349088581856, 0.015099559152917138, 0.024014605321403704, 0.03998781056232702, 0.002137977541720041, 0.0015413006453101077, 0.02021349088581856, 0.015099559152917138, 0.05576200449386415, 0.002707773332090879, 0.016954307098411187, 0.0015413006453101077, 0.0028187403977459127, 0.016650712285423714, 0.016650712285423714, 0.012709845912

In [201]:
data_vec.shape

(17, 208)

Задание 2.
найти фичи с наибольшей значимостью, и вывести их

In [202]:
# сортируем по убыванию 10 слов с максимальным средним TF-IDF
mean_weights_df.sort_values(by = 'mean_weights', ascending = False).reset_index(drop = True).head(10)

,term,mean_weights
0,не,0.105020
1,то,0.055812
2,за,0.055762
3,тебя,0.046921
4,кто,0.045198
5,так,0.044139
6,ты,0.043119
7,он,0.042968
8,же,0.039988
9,все,0.038878


In [203]:
#токины высокой частотности
df_h = pd.DataFrame(columns = ['term', 'wieght'])
df_h = mean_weights_df.loc[mean_weights_df['mean_weights'] > 0.05]
df_h.head()
h_list = df_h['term'].to_list()
h_list

['за', 'не', 'то']

In [204]:
#токины средней частотности
df_m = pd.DataFrame(columns = ['term', 'wieght'])
df_m = mean_weights_df.loc[(mean_weights_df['mean_weights'] > 0.03) & (mean_weights_df['mean_weights'] <= 0.05)]
m_list = df_m['term'].to_list()
m_list

['все',
 'выздоравливаю',
 'же',
 'кто',
 'нет',
 'но',
 'ну',
 'он',
 'так',
 'тебя',
 'ты',
 'что']

In [205]:
#токины низкой частотности
df_l = pd.DataFrame(columns = ['term', 'wieght'])
df_l = mean_weights_df.loc[mean_weights_df['mean_weights'] <= 0.03]
l_list = df_l['term'].to_list()
l_list

['авторы',
 'блин',
 'бля',
 'болею',
 'болеюа',
 'буду',
 'будужелаю',
 'был',
 'былна',
 'было',
 'ведь',
 'взяли',
 'винды',
 'виндыколлеги',
 'вообще',
 'вообщеа',
 'вот',
 'вседа',
 'вчера',
 'вчеракот',
 'говорят',
 'года',
 'голода',
 'дада',
 'долбанной',
 'долбаннойколлеги',
 'долго',
 'друзей',
 'есть',
 'ещ',
 'ждут',
 'желаюжелаю',
 'жена',
 'жр',
 'жрать',
 'заколлеги',
 'закрытие',
 'закрытиена',
 'заобновил',
 'затихарился',
 'зашли',
 'значит',
 'знаю',
 'знаюну',
 'идиотка',
 'из',
 'изколлеги',
 'или',
 'илину',
 'испугалась',
 'каждое',
 'каждоена',
 'как',
 'каким',
 'какимобновил',
 'кирпичами',
 'кирпичамия',
 'когда',
 'коллегиколлеги',
 'конечно',
 'коткот',
 'ктону',
 'курили',
 'лешим',
 'лешимобновил',
 'лучше',
 'любишь',
 'любишьну',
 'мальчик',
 'мальчикда',
 'месяца',
 'месяцана',
 'микрофраза',
 'микрофразая',
 'мимими',
 'много',
 'могу',
 'мои',
 'мой',
 'мойда',
 'молчит',
 'мы',
 'на',
 'нада',
 'нана',
 'нас',
 'неа',
 'него',
 'негода',
 'нее',
 'н

In [206]:
mask = data['text'].apply(lambda x: any(word in x.lower() for word in h_list))
h_text = data[mask]
h_text = h_text.reset_index(drop=True)
h_text.head(3)

,text,label
0,хоть хоть я и школота но поверь у нас то же...,1
1,дада все таки он немного похож на него но мо...,1
2,ну ну ты идиотка я испугалась за тебя ...,1


In [207]:
h_text.shape

(14, 2)

In [208]:
mask = data['text'].apply(lambda x: any(word in x.lower() for word in m_list))
m_text = data[mask]
m_text = m_text.reset_index(drop=True)
m_text.head(3)

,text,label
0,хоть хоть я и школота но поверь у нас то же...,1
1,дада все таки он немного похож на него но мо...,1
2,ну ну ты идиотка я испугалась за тебя ...,1


In [209]:
m_text.shape

(17, 2)

In [210]:
mask = data['text'].apply(lambda x: any(word in x.lower() for word in l_list))
l_text = data[mask]
l_text = l_text.reset_index(drop=True)
l_text.head(3)

,text,label
0,хоть хоть я и школота но поверь у нас то же...,1
1,дада все таки он немного похож на него но мо...,1
2,ну ну ты идиотка я испугалась за тебя ...,1


In [211]:
l_text.shape

(17, 2)

In [212]:
#классификация на основе высокочастотных токенов
from sklearn import model_selection, preprocessing, linear_model
from sklearn.feature_extraction.text import CountVectorizer

# labelEncode целевую переменную
X_train, X_test, y_train, y_test = model_selection.train_test_split(h_text['text'], h_text['label'])

encoder = preprocessing.LabelEncoder()
y_train = encoder.fit_transform(y_train)
y_test = encoder.fit_transform(y_test)

vectorizer = TfidfVectorizer()
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)


classifier = SVC()
classifier.fit(X_train_tfidf, y_train)

predictions = classifier.predict(X_test_tfidf)
predictions

array([0, 1, 1, 1])

In [213]:
X_train_tfidf.shape

(10, 121)

In [214]:
from sklearn.metrics import accuracy_score

In [215]:
accuracy_score(y_test, predictions)

0.5

In [216]:
metr = pd.DataFrame(columns = ['частота', 'accuracy_score'])
metr.loc[len(metr.index)] = ['высокая', accuracy_score(y_test, predictions)]
metr

,частота,accuracy_score
0,высокая,0.5


In [217]:
#классификация на основе токенов средней частоты
X_train, X_test, y_train, y_test = model_selection.train_test_split(m_text['text'], m_text['label'])

encoder = preprocessing.LabelEncoder()
y_train = encoder.fit_transform(y_train)
y_test = encoder.fit_transform(y_test)

vectorizer = TfidfVectorizer()
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

classifier = SVC()
classifier.fit(X_train_tfidf, y_train)

predictions = classifier.predict(X_test_tfidf)
predictions

array([0, 0, 0, 0, 0])

In [218]:
X_train_tfidf.shape

(12, 150)

In [219]:
accuracy_score(y_test, predictions)

0.6

In [220]:
metr.loc[len(metr.index)] = ['средняя', accuracy_score(y_test, predictions)]
metr

,частота,accuracy_score
0,высокая,0.5
1,средняя,0.6


In [221]:
from sklearn.svm import SVC
#классификация на основе токенов нижкой частоты
X_train, X_test, y_train, y_test = model_selection.train_test_split(l_text['text'], l_text['label'])

encoder = preprocessing.LabelEncoder()
y_train = encoder.fit_transform(y_train)
y_test = encoder.fit_transform(y_test)

vectorizer = TfidfVectorizer()
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

classifier = SVC()
classifier.fit(X_train_tfidf, y_train)

predictions = classifier.predict(X_test_tfidf)
predictions

array([0, 0, 0, 0, 0])

In [222]:
X_test_tfidf.shape

(5, 157)

In [223]:
accuracy_score(y_test, predictions)

0.2

In [224]:
metr.loc[len(metr.index)] = ['низкая', accuracy_score(y_test, predictions)]
metr

,частота,accuracy_score
0,высокая,0.5
1,средняя,0.6
2,низкая,0.2


Задание 3.
1) сравнить count/tf-idf/hashing векторайзеры/полносвязанную сетку (построить classification_report)

2) подобрать оптимальный размер для hashing векторайзера

3) убедиться что для сетки нет переобучения

In [225]:
metr_new = pd.DataFrame(columns = ['векторайзер', 'accuracy_score'])

In [226]:
# CountVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
# Разделение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = model_selection.train_test_split(data['text'], data['label'])#, test_size=0.2, random_state=42)

encoder = preprocessing.LabelEncoder()
y_train = encoder.fit_transform(y_train)
y_test = encoder.fit_transform(y_test)

# Инициализация и применение CountVectorizer для преобразования текстовых данных в векторы признаков
vectorizer = CountVectorizer()
X_train_counts = vectorizer.fit_transform(X_train)
X_test_counts = vectorizer.transform(X_test)

# Инициализация и обучение модели логистической регрессии
classifier = LogisticRegression()
classifier.fit(X_train_counts, y_train)

predictions = classifier.predict(X_test_counts)
accuracy_score(y_test, predictions)

metr_new.loc[len(metr_new.index)] = ['CountVectorizer', accuracy_score(y_test, predictions)]
metr_new

,векторайзер,accuracy_score
0,CountVectorizer,0.6


In [227]:
# CountVectorizer
# Разделение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = model_selection.train_test_split(data['text'], data['label'])#, test_size=0.2, random_state=42)

encoder = preprocessing.LabelEncoder()
y_train = encoder.fit_transform(y_train)
y_test = encoder.fit_transform(y_test)

# Инициализация и применение CountVectorizer для преобразования текстовых данных в векторы признаков
vectorizer = TfidfVectorizer()
X_train_tv = vectorizer.fit_transform(X_train)
X_test_tv = vectorizer.transform(X_test)

# Инициализация и обучение модели логистической регрессии
classifier = LogisticRegression()
classifier.fit(X_train_tv, y_train)

predictions = classifier.predict(X_test_tv)
accuracy_score(y_test, predictions)

metr_new.loc[len(metr_new.index)] = ['TfidfVectorizer', accuracy_score(y_test, predictions)]
metr_new

,векторайзер,accuracy_score
0,CountVectorizer,0.6
1,TfidfVectorizer,0.2


In [228]:
#HashingVectorizer()
from sklearn.feature_extraction.text import HashingVectorizer

X_train, X_test, y_train, y_test = model_selection.train_test_split(data['text'], data['label'])#, test_size=0.2, random_state=42)

encoder = preprocessing.LabelEncoder()
y_train = encoder.fit_transform(y_train)
y_test = encoder.fit_transform(y_test)

# Инициализация и применение HashingVectorizer для преобразования текстовых данных в векторы признаков
vectorizer = HashingVectorizer()
X_train_hv = vectorizer.fit_transform(X_train)
X_test_hv = vectorizer.transform(X_test)

# Инициализация и обучение модели логистической регрессии
classifier = LogisticRegression()
classifier.fit(X_train_hv, y_train)

predictions = classifier.predict(X_test_hv)
accuracy_score(y_test, predictions)

metr_new.loc[len(metr_new.index)] = ['HashingVectorizer', accuracy_score(y_test, predictions)]
metr_new


,векторайзер,accuracy_score
0,CountVectorizer,0.6
1,TfidfVectorizer,0.2
2,HashingVectorizer,0.4


In [229]:
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.losses import SparseCategoricalCrossentropy

X_train, X_test, y_train, y_test = train_test_split(data['text'], data['label'])
vectorizer = CountVectorizer()
X_train_counts = vectorizer.fit_transform(X_train)
X_test_counts = vectorizer.transform(X_test)
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

# Создание модели многослойной нейронной сети
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(X_train_counts.shape[1],)))
model.add(Dense(32, activation='relu'))
model.add(Dense(2, activation='softmax'))

model.compile(optimizer='adam',
              loss=SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.fit(X_train_counts.toarray(), y_train_encoded, epochs=10, batch_size=32, verbose=1)

_, accuracy = model.evaluate(X_test_counts.toarray(), y_test_encoded, verbose=0)
print(f"Точность модели: {accuracy}")

predictions = model.predict(X_test_counts.toarray())

# Вывод предсказанных меток классов
predicted_labels = label_encoder.inverse_transform(predictions.argmax(axis=1))

metr_new.loc[len(metr_new.index)] = ['NN', accuracy]
metr_new

Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/backend.py:5714: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


1/1 [==============================] - 1s 729ms/step - loss: 0.7422 - accuracy: 0.5833
Epoch 2/10
1/1 [==============================] - 0s 10ms/step - loss: 0.4840 - accuracy: 0.6667
Epoch 3/10
1/1 [==============================] - 0s 11ms/step - loss: 0.3320 - accuracy: 0.8333
Epoch 4/10
1/1 [==============================] - 0s 12ms/step - loss: 0.2439 - accuracy: 0.8333
Epoch 5/10
1/1 [==============================] - 0s 9ms/step - loss: 0.1865 - accuracy: 0.8333
Epoch 6/10
1/1 [==============================] - 0s 10ms/step - loss: 0.1455 - accuracy: 1.0000
Epoch 7/10
1/1 [==============================] - 0s 11ms/step - loss: 0.1143 - accuracy: 1.0000
Epoch 8/10
1/1 [==============================] - 0s 12ms/step - loss: 0.0915 - accuracy: 1.0000
Epoch 9/10
1/1 [==============================] - 0s 10ms/step - loss: 0.0734 - accuracy: 1.0000
Epoch 10/10
1/1 [==============================] - 0s 11ms/step - loss: 0.0592 - accuracy: 1.0000
Точность модели: 1.0
1/1 [==============

,векторайзер,accuracy_score
0,CountVectorizer,0.6
1,TfidfVectorizer,0.2
2,HashingVectorizer,0.4
3,NN,1.0
